In [3]:
## Task 1. Install packages and import libraries
!pip install -U langchain-community langchain_google_vertexai google-cloud-aiplatform langchain langchain-chroma langchain-community langchain-core langchain-google-vertexai langchain-text-splitters pypdf 

  Using cached langchain_community-0.3.5-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_google_vertexai-2.0.7-py3-none-any.whl.metadata (3.8 kB)
  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [4]:
### Install Vertex AI SDK & Other dependencies

!sudo apt -y -qq install tesseract-ocr
!sudo apt -y -qq install libtesseract-dev
!sudo apt-get -y -qq install poppler-utils #required by PyPDF2 for page count and other pdf utilities
!sudo apt-get -y -qq install python-dev libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

The following additional packages will be installed:
  liblept5 libopenjp2-7 libtesseract4 libwebpmux3 tesseract-ocr-eng
  tesseract-ocr-osd
The following NEW packages will be installed:
  liblept5 libopenjp2-7 libtesseract4 libwebpmux3 tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 7 newly installed, 0 to remove and 3 not upgraded.
Need to get 7483 kB of archives.
After this operation, 23.2 MB of additional disk space will be used.

78Selecting previously unselected package libopenjp2-7:amd64.
(Reading database ... 138836 files and directories currently installed.)
Preparing to unpack .../0-libopenjp2-7_2.4.0-3_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [  3%] [##........................................................] 8Unpacking libopenjp2-7:amd64 (2.4.0-3) ...
7Progress: [  7%] [####......................................................] 8Selecting previously unselected package libwebpmux3:amd64.

In [5]:
# # Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()

app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [4]:
# Import libraries

import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain_google_vertexai import ChatVertexAI

warnings.filterwarnings("ignore")
from langchain.chains import (
ConversationChain,
LLMChain,
RetrievalQA,
SimpleSequentialChain,
)
from langchain.chains.summarize import load_summarize_chain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_google_vertexai import ChatVertexAI, VertexAI, VertexAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
## Task 2. Load the model and prepare data


File saved to: data/practitioners_guide_to_mlops_whitepaper.pdf


In [6]:
#TODO: Load the pre-trained text generation model named gemini-pro using ChatVertexAI.
#TODO: Load the pre-trained text generation model named gemini-pro using ChatVertexAI.
# LLM model
llm = VertexAI(
model_name="gemini-1.5-flash",
verbose=True,
)
# Chat
chat = ChatVertexAI(model="gemini-pro")

# Embedding
embeddings = VertexAIEmbeddings("text-embedding-004")



In [3]:
#TODO: Download a PDF file from specified URL and save it in "data" directory
import os
from pathlib import Path
import urllib.request

# Define the URL of the file to download
url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"

# Create a local directory named "data"
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist

# Define the path where the file will be saved
file_path = data_dir / "practitioners_guide_to_mlops_whitepaper.pdf"

# Download and save the file
urllib.request.urlretrieve(url, file_path)

print(f"File saved to: {file_path}")

File saved to: data/practitioners_guide_to_mlops_whitepaper.pdf


In [7]:
#TODO: Load the PDF file and split it into individual pages.
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

# Define the path to the PDF file
pdf_file_path = Path("data/practitioners_guide_to_mlops_whitepaper.pdf")

# Create an instance of the PyPDFLoader
loader = PyPDFLoader(pdf_file_path)

# Load and split the PDF into individual pages
documents = loader.load_and_split()

# Print the number of pages loaded
print(f"Number of pages loaded: {len(documents)}")

# Optionally, print the content of the first page
if documents:
    print("Content of the first page:")
    print(documents[0].page_content)


Number of pages loaded: 36
Content of the first page:
Practitioners guide to MLOps: 
A framework for continuous 
delivery and automation of 
machine learning.
White paper
May 2021
Authors: 
Khalid Salama, 
Jarek Kazmierczak, 
Donna Schut


In [9]:
## Task 3. Generate summaries

In [10]:
#Prompt design with Stuffing Chain

prompt_template = """Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
  """

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

In [11]:
#TODO: Set up a summarization chain using the stuff method. Incorporate the model loaded earlier into the summarization chain to enhance the quality of the summary.
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Load the summarize chain with "stuff" as the chain type
chain = load_summarize_chain(llm, chain_type="stuff", verbose=True)

# Run the chain on the split texts
summary = chain.run(texts)

# Print the summary
print(summary)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Practitioners guide to MLOps: 
A framework for continuous 
delivery and automation of 
machine learning.
White paper
May 2021
Authors: 
Khalid Salama, 
Jarek Kazmierczak, 
Donna Schut

Table of Contents
Executive summary 3
Overview of MLOps lifecycle and core capabilities 4
Deep dive of MLOps processes 15
Putting it all together 34
Additional resources 36
Building an ML-enabled system 6
The MLOps lifecycle 7
MLOps: An end-to-end workflow 8
MLOps capabilities 9
      Experimentation 11
      Data processing 11
      Model training 11
      Model evaluation 12
      Model serving 12
      Online experimentation 13
      Model monitoring 13
      ML pipelines 13
      Model registry 14
      Dataset and feature repository 14
      ML metadata and artifact tracking 15
ML development 16
Training operationalization 18
Continuous training 20
Mo

In [ ]:
## Task 4. Add tools to the LLM for multiplication


In [55]:
#TODO: Create a tool called multiply which takes in two integers a and b and performs a multiplication (a*b) returning the result.
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers."""
    return a * b



In [60]:
#TODO: Add the tools to the LLM created earlier and invoke it with the following query. Print the result in the console.
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
query = "What is 3 * 12?"
messages = [HumanMessage(query)]
# response=[ToolMessage(tool_name="multiply",tool_args=(3,12),content="Multiplication task",tool_call_id="multiply_1")]
response = chat(messages)
print(response)

content='3 multiplied by 12 is 36.' additional_kwargs={} response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}], 'usage_metadata': {'prompt_token_count': 9, 'candidates_token_count': 11, 'total_token_count': 20, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.36905228007923474, 'logprobs_result': {'top_candidates': [], 'chosen_candidates': []}} id='run-7d39a1cf-39cf-47b3-8247-5b42197d94d1-0' usage_metadata={

In [62]:
#TODO: Iterate through the tools in the response, invoke the tools and append the response to the messages object.
# Step 5: Iterate through the tools in the response
for message in response:
    if isinstance(message, ToolMessage):
        # Extract the tool name and arguments
        tool_name = message.tool_name
        tool_args = message.tool_args
        print(tool_name)
        # Invoke the tool
        if tool_name == "multiply":
            result = multiply(*tool_args)  # Unpack arguments for the multiply function
            # Append the tool response to messages
            messages.append(AIMessage(f"The result of {tool_args[0]} * {tool_args[1]} is {result}."))

# Step 6: Prepare the final prompt from messages
final_prompt = "\n".join([msg.content for msg in messages])  # Concatenate messages into a single string

In [63]:
#TODO: Invoke the LLM with the solution of the tool and the original message and print the final user response.
final_response = llm(final_prompt)

# Step 8: Print the final user response
print(final_response)


3 * 12 = 36 

